# Waveform Visualization Notebook

Set `VIDEO_PATH` to your local video. This notebook uses the same helpers as the API to:
- Decode video frames
- Build a raw green-channel trace (skin-masked)
- Band-pass filter (broad) to estimate HR (FFT)
- HR-anchored narrow band-pass to reveal pulse waveform structure
- Plot raw vs. filtered traces over time (seconds)

Tip: Keep clips ≥10–20s for clearer spectra.


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from app.services.video_io import video_to_frames
from app.services.signal_proc import (
    emphasize_green_mean_rgb,
    preprocess_bvp,
    preprocess_bvp_anchored,
    bpm_fft,
)
from app.config import FPS

# ---- User input ----
VIDEO_PATH = os.getenv("VIZ_VIDEO", "./sample.mp4")  # set to your file
# --------------------

# Read video bytes
with open(VIDEO_PATH, "rb") as f:
    data = f.read()

# Extract frames and fs using the same API helper
frames, fs_used, enh_used, crop_used, _ = video_to_frames(data)
fs = int(fs_used) if fs_used else FPS

print(f"Frames: {len(frames)} | fs: {fs} | enhancement: {enh_used} | face_crop: {crop_used}")

# Raw green-trace (skin masked)
raw_green = emphasize_green_mean_rgb(frames)

# Broad band-pass and HR estimate
bvp_broad = preprocess_bvp(raw_green, fs)
hr_est = bpm_fft(bvp_broad, fs)
print(f"FFT HR estimate: {hr_est:.1f} bpm")

# HR-anchored narrow band-pass to reveal pulse waveform
bvp_anchored = preprocess_bvp_anchored(raw_green, fs, hr_est)

t = np.arange(len(raw_green)) / fs

plt.figure(figsize=(12, 6))
plt.plot(t, raw_green - np.mean(raw_green), label="Raw green (centered)", alpha=0.5)
plt.plot(t, bvp_broad - np.mean(bvp_broad), label="Broad band-pass", linewidth=1.5)
plt.plot(t, bvp_anchored - np.mean(bvp_anchored), label="HR-anchored band-pass", linewidth=2)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (a.u.)")
plt.title("rPPG Waveform Structure")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
